In [1]:
import cv2
import serial
import time
import numpy as np 
import math

ser = serial.Serial('COM3', 115200, timeout=0.002)
time.sleep(3)


ser.write(str(44).encode())
time.sleep(1)

ser.write(str(64).encode())
time.sleep(1)

ser.write(str(66).encode())
time.sleep(1)

ser.write(str(46).encode())
time.sleep(1)

ser.write(str(55).encode())
  

2

In [2]:
def findMeanBrightness(img):
    img[img < 120] = 0
    a0 = np.sum(img, axis=0)
    a0 = a0-np.min(a0)
    a1 = np.sum(img, axis=1)
    a1 = a1 - np.min(a1)
    bigsum = np.sum(a0)
    weights = np.arange(zoom*2)
    x = np.sum(a0 * weights) / bigsum
    y = np.sum(a1 * weights) / bigsum
    if math.isnan(x) or math.isnan(y):
        x = zoom
        y = zoom
    return y-zoom, x-zoom
    
# Setting constants
cap = cv2.VideoCapture(0) 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
frames = []
pos = []
posHistory = []
k=-1
zoom = 80
a = None
move = False

# Main loop
t0 = time.time()
while True:
    # Read new frame
    ret, frame = cap.read()
    frame = np.array(frame[:, 300:-300, 2], dtype=np.uint8)
    
    # Find a and b (coords of ball)
    if k == 32 or a is None:
        a, b = frame.max(0).argmax(), frame.max(1).argmax()
        for i in range(15):
            posHistory.append([a, b])
    crop = frame[int(b)-zoom:int(b)+zoom, int(a)-zoom:int(a)+zoom].copy()
    w, h = frame.shape[:2]
    
    # Fine tune the position with old code
    if crop.shape[:2] == (zoom*2, zoom*2):
        biasB, biasA = findMeanBrightness(crop)
        a+=biasA
        b+=biasB
        crop[int(biasB)] = 0
        crop[:, int(biasA)] = 0
        cv2.imshow("Crop", cv2.flip(cv2.flip(crop, 1), 0))
    
    # Show the position
    frame[int(b)] = 0
    frame[:, int(a)] = 0
    frame[int(w*0.4)] = 150
    frame[int(w*0.6)] = 150
    frame[:, int(h*0.4)] = 150
    frame[:, int(h*0.6)] = 150
    
    # Save the position for later
    if len(posHistory) < 3:
        for i in range(25):
            posHistory.append([a, b])
    posHistory.append([a, b])
    
    # Find future position
    command = 0
    # Get the derivatives
    diff = time.time()-t0
    print(diff)
    t0 = time.time()
    pos  = np.array(posHistory[-8:])
    vel  = (pos[1:] - pos[:-1])*diff*10
    acc  = (vel[1:] - vel[:-1])*diff*10
    jerk = (acc[1:] - acc[:-1])*diff*10
    memory = 4
    derivatives = [pos[-memory:].mean(0), vel[-memory:].mean(0), acc[-memory:].mean(0), jerk[-memory:].mean(0)]
    visuals = [np.zeros((200, 200)) for i in range(4)]
    for v, d in zip(visuals, derivatives):
        v = cv2.circle(v, (100+int(d[0]*5), 100+int(d[1]*5)), 10, 255, -1)
    for i in range(4):
        cv2.imshow(f"D{i}", visuals[i])
    # Make a simulation
    sp, sv, sa, _ = derivatives
    frame = np.array(frame, dtype=np.uint8)
    spHistory = []
    for i in range(90):
        sp += sv 
        if i < 70:
            sv += sa*0.5
        spHistory.append(sp)
        cv2.circle(frame, (int(sp[0]), int(sp[1])), 10, min(i*4, 255), 1)
    
    #na, nb = np.array(np.array(posHistory[-1])*5-np.array(posHistory[-5])*4)
    na, nb = spHistory[-1]
    #print(na, nb, sp)
    
    
    # Control the motors
    if na < h*0.4:
        command += 60
    elif na > h*0.6:
        command += 40
    else:
        command += 50
    if nb < w*0.4:
        command += 6
    elif nb > w*0.6:
        command += 4
    else:
        command += 5
    
    controlSystem = np.zeros((200, 200, 3))
    cv2.circle(controlSystem, (200-int(math.floor(command/10)-4)*100, 200-int(command%10-4)*100), 10, (200, 0, 0), 4)
    cv2.circle(controlSystem, (int(min(max((na/w), 0), 1)*200), int(min(max((nb/h), 0), 1)*200)), 10, (0, 200, 0), 4)
    cv2.imshow("ControlSystem", controlSystem)
    
    # Send message
    if len(str(command)) == 2 and k != 32 and move:
        ser.write(str(command).encode()) # Command must be 4-6
        print(command)
    # Debug View
    frame = cv2.circle(np.array(frame, dtype=np.uint8), (int(na), int(nb)), 10, 255, -1)
    cv2.imshow("Webcam", cv2.flip(cv2.flip(frame, 1), 0))
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
    if k == ord("m"):
        move = True
    if k == ord("s"):
        move = False
        ser.write(str(55).encode())
        
ser.write(str(55).encode())
cap.release()
cv2.destroyAllWindows()




0.3148059844970703
0.15556812286376953
0.022007226943969727
0.020004749298095703
0.020003557205200195
0.02100825309753418
0.01951742172241211
0.020004749298095703
0.017003536224365234
0.02100539207458496
0.03106856346130371
0.04894399642944336
0.03188967704772949
0.048026084899902344
0.018004179000854492
0.018004179000854492
0.04463958740234375
0.031473398208618164
0.03259134292602539
0.048316001892089844
